In [0]:
from pyspark.sql.functions import col

In [0]:
%run "../includes/common_functions"

In [0]:
latest_update = spark.sql("""select * 
                          from cars_catalog.default.metadata_table
                          where table_name = 'dim_dealer' 
                          """).collect()[0][1]
latest_update

datetime.datetime(2025, 5, 6, 14, 56, 3, 186000)

In [0]:
dim_table = spark.read.format("delta") \
                        .load("abfss://gold@cardeprojectdl.dfs.core.windows.net/dim_dealer") \
                        .filter(col("updated_at") == latest_update)

In [0]:
dim_table.display()

Dealer_ID,DealerName,updated_at,dim_dealer_key
DLR0062,Ford Australia Motors,2025-05-06 14:56:03.186000,71
DLR0063,Ford do Brasil Motors,2025-05-06 14:56:03.186000,233
DLR0060,"Fisker, Karma Motors",2025-05-06 14:56:03.186000,137
XYZ0063,Datafam Dealers,2025-05-06 14:56:03.186000,268


In [0]:
sql_create_table = """
    CREATE TABLE IF NOT EXISTS CARSALES_PROJECT.DW.dim_dealer (
        DealerName STRING,
        Dealer_ID STRING,
        dim_dealer_key NUMBER,
        updated_at TIMESTAMP
    )

"""

In [0]:
sql_merging = """
            MERGE INTO CARSALES_PROJECT.DW.dim_dealer AS tar
            USING CARSALES_PROJECT.EXTERNAL_STAGE.STG_DIM_DEALER AS src
            ON tar.dim_dealer_key = src.dim_dealer_key
            WHEN MATCHED THEN
                UPDATE SET tar.DealerName = src.DealerName,
                            tar.Dealer_ID = src.Dealer_ID,
                            tar.updated_at = src.updated_at
            WHEN NOT MATCHED THEN   
                INSERT (Dealer_ID, DealerName, dim_dealer_key, updated_at) VALUES (src.Dealer_ID, src.DealerName, src.dim_dealer_key, src.updated_at)
"""

In [0]:
write_to_snowflake(dim_table,"STG_DIM_DEALER",sql_create_table,sql_merging)